## Loading dependencies

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/MachineLearningProject")
import sys
sys.path.append("/content/drive/MyDrive/MachineLearningProject")

Mounted at /content/drive


## Creating the model

In [ ]:
%run sequential.py

## Loading the data and merging the datasets

In [ ]:
def load_images_and_labels(folder_path, label):
    images = []
    labels = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
            labels.append(label)

    return np.array(images), np.array(labels)

In [ ]:
train_muffin_images, train_muffin_labels = load_images_and_labels("/content/drive/MyDrive/MachineLearningProject/data/train/muffin", label=1)
train_chihuahua_images, train_chihuahua_labels = load_images_and_labels("/content/drive/MyDrive/MachineLearningProject/data/train/chihuahua", label=0)
test_muffin_images, test_muffin_labels = load_images_and_labels("/content/drive/MyDrive/MachineLearningProject/data/test/muffin", label=1)
test_chihuahua_images, test_chihuahua_labels = load_images_and_labels("/content/drive/MyDrive/MachineLearningProject/data/test/chihuahua", label=0)

images = np.concatenate([train_muffin_images, train_chihuahua_images, test_muffin_images, test_chihuahua_images])
labels = np.concatenate([train_muffin_labels, train_chihuahua_labels, test_muffin_labels, test_chihuahua_labels])

## Cross-validation

In [ ]:
zero_one_loss = []

k= 5
kf = KFold(n_splits=k,
           shuffle=True)


for train_index, test_index in kf.split(images):


      train_images, val_images = images[train_index], images[test_index]
      train_labels, val_labels = labels[train_index], labels[test_index]

      # Loading the model
      seq_2 = tf.keras.models.load_model(
    "/content/drive/MyDrive/MachineLearningProject/models/sequential/seq_2.keras",
    safe_mode=False)

      # Compiling the model
      seq_2.compile(loss="binary_crossentropy",
              optimizer=Adam(learning_rate=0.0005),
              metrics=["accuracy"])

      # Fitting the model on the training folds
      seq_2_history = seq_2.fit(train_images, train_labels,
                                validation_data=(val_images, val_labels),
                                epochs=50)

      # Evaluating the model on the test fold
      _, accuracy = seq_2.evaluate(val_images, val_labels)

      # Calculating the zero-one loss
      zero_one_loss.append(1 - accuracy)


Epoch 1/50
149/149 [==============================] - 18s 63ms/step - loss: 0.4032 - accuracy: 0.8182 - val_loss: 0.3968 - val_accuracy: 0.8255
Epoch 2/50
149/149 [==============================] - 8s 54ms/step - loss: 0.2998 - accuracy: 0.8772 - val_loss: 0.3286 - val_accuracy: 0.8592
Epoch 3/50
149/149 [==============================] - 8s 54ms/step - loss: 0.2577 - accuracy: 0.8914 - val_loss: 0.3372 - val_accuracy: 0.8744
Epoch 4/50
149/149 [==============================] - 8s 54ms/step - loss: 0.2221 - accuracy: 0.9093 - val_loss: 0.2703 - val_accuracy: 0.8946
Epoch 5/50
149/149 [==============================] - 8s 54ms/step - loss: 0.2068 - accuracy: 0.9161 - val_loss: 0.2648 - val_accuracy: 0.9022
Epoch 6/50
149/149 [==============================] - 8s 54ms/step - loss: 0.1936 - accuracy: 0.9232 - val_loss: 0.2393 - val_accuracy: 0.9039
Epoch 7/50
149/149 [==============================] - 8s 54ms/step - loss: 0.1815 - accuracy: 0.9317 - val_loss: 0.2667 - val_accuracy: 0.888

In [ ]:
print(zero_one_loss)
accuracies = np.ones(5) - zero_one_loss
print("The mean zero-one losse is:")
print(round(np.mean(zero_one_loss), 4))
print("The mean accuracy is:")
print(round(np.mean(accuracies),4))

0.0535
0.9465


In [ ]:
accuracies = [round(elem, 4) for elem in accuracies]
zero_one_loss = [round(elem, 4) for elem in zero_one_loss]
print("The zero-one losses are:")
print(zero_one_loss)
print("The accuracies are:")
print(accuracies)

The zero-one losses are:
[0.0725, 0.0481, 0.0473, 0.0338, 0.0658]
The accuracies are:
[0.9275, 0.9519, 0.9527, 0.9662, 0.9342]
